In [1]:
from osgeo import ogr
from osgeo import gdal
import numpy as np
import pandas as pd
import geopandas as gpd
import critical_loads as cl
import nivapy

# Connect to db
engine = nivapy.da.connect()

Username: ········
Password: ········
Connection successful.


# Critical Loads Workflow

This notebook implements a new workflow for the Critical Loads processing. There are three main components: vegetation, water and soils.

## 1. Vegetation

A new raster-based workflow was developed in [this notebook](http://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_vegetation.ipynb). The code below loops over all the datasets to create vegetation exceedance grids (in mg-N/l) for the time periods of interest.

### 1.1. Upload new data to database

In [2]:
# ID for new series
ser_id = 26

# Create new series for this data period
df = pd.DataFrame({'dep_series_id':ser_id,
                   'name':'Middel 2012-2016',
                   'description':'Fordelt til BLR av NILU 2017 (Wenche Aas; old method)'},
                  index=[0,])

## Add to db
#df.to_sql('dep_series_definitions', con=engine, schema='resa2', 
#          if_exists='append', index=False)

In [3]:
# Read data
in_xlsx = (ur'K:\Avdeling\317 Klima- og miljømodellering\KAU\Focal Centre'
           ur'\Projects\2017 Tålegrenseprosjekt\Data\dep1216_til_NIVA_old method_update.xlsx')
df = pd.read_excel(in_xlsx, sheetname='16x16 NIVA', index_col=0)
df.index.name = 'blr'

# Map headings to db par IDs
par_dict = {'atnsss1216':4,  # Non-marine S
            'atnox1216':1,   # N (oks)
            'atnhx1216':2,   # N (red)
            'ats1216':3}     # Total S

# Get cols and rename
df = df[par_dict.keys()]
df.columns = [par_dict[i] for i in df.columns]
df.reset_index(inplace=True)

# Melt 
df = pd.melt(df, var_name='parameter_id', id_vars='blr')

# Add series ID
df['dep_series_id'] = ser_id

## Add to db
#df.to_sql('dep_blr_values', con=engine, schema='resa2', 
#          if_exists='append', index=False)

### 1.2. Read lookup table link vegetation classes to critical loads

In [4]:
# Read lookup table
in_xlsx = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\vegetation'
           r'\sat_veg_land_use_classes.xlsx')
df = pd.read_excel(in_xlsx, sheetname='EUNIS_tilGIS', index_col=0)

df = df[['CL_100smgN/m2/yr']].round(0).astype(int)

df.head()

,CL_100smgN/m2/yr
NORUTcode,
1,5
2,5
3,5
4,10
5,10


### 1.3. Reclassify

In [5]:
# Input national veg map
in_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
          r'\GIS\Raster\sat_veg_30m_all.tif')

# Output geotiff for critical loads values
out_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
           r'\GIS\Raster\sat_veg_30m_cr_lds_div100.tif')

# Mask raster for land defined by BLR
mask_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
            r'\GIS\Raster\blr_land_mask.tif')

# Reclassify
cl.reclassify_raster(in_tif, mask_tif, out_tif, df, 'CL_100smgN/m2/yr', 255)

### 1.4. Get deposition data from database

**Note:** Remember to add the new `dep_series_id` to the query below, as well as a new name in `df.columns`.

In [6]:
# Get all dep values
sql = ("SELECT blr, dep_series_id as series, value as dep "
       "FROM resa2.dep_blr_values "
       "WHERE parameter_id IN (1, 2) "
       "AND dep_series_id IN (1, 2, 3, 4, 25, 26)")

df = pd.read_sql(sql, engine)

# Sum N_oks and N_red
df = df.groupby(['blr', 'series']).sum()

# Reshape and tidy
df = df.unstack()
df.columns = df.columns.get_level_values(1)
df.columns.name = ''
df.columns = ['Ndep78_82', 'Ndep92_96', 'Ndep97_01', 'Ndep02_06', 
              'Ndep07_11', 'Ndep12_16']

print np.nanmin(df.values), np.nanmax(df.values)

df.head()

69.8 2252.72


,Ndep78_82,Ndep92_96,Ndep97_01,Ndep02_06,Ndep07_11,Ndep12_16
blr,,,,,,
58006001,1700.00,1771.17,1576.91,NaN,1363.48,1514.08
58006002,1728.85,1247.05,1149.28,1192.63,1383.95,1406.96
58006003,1217.42,1171.20,1059.42,1157.39,1203.15,1228.37
58006004,1392.29,1310.32,1425.35,1302.15,1112.38,1177.41
58006005,1293.28,1302.73,1255.17,1037.57,1184.53,1286.92


### 1.5. Loop over datasets

In [7]:
%%time

# Path to raw BLR shapefile
in_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
          r'\Shapefiles\blrgrid_uten_grums_utm_z33n.shp')

# Snap raster
snap_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
            r'\GIS\Raster\sat_veg_30m_all.tif')

# Critical loads raster
cl_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
          r'\GIS\Raster\sat_veg_30m_cr_lds_div100.tif')

# Container for output
data_dict = {'series':[],
             'nor_area_km2':[],
             'ex_area_km2':[]}

# Loop over series
for ser in df.columns:    
    
    print 'Processing: %s' % ser
    print '    Building deposition shapefile...'
    
    # Get deposition
    dep_df = df[[ser]].dropna(how='any').round(0).astype(int).reset_index()

    # Rename cols to match shapefile
    dep_df.columns = ['BLR', ser]

    # Read shapefile
    blr_df = gpd.read_file(in_shp)

    # Join and tidy
    dep_df = blr_df.merge(dep_df, on='BLR')
    del dep_df['area_m2']
    
    # Write output shapefile
    dep_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
               r'\Shapefiles\dep_%s.shp' % ser)
    dep_df.to_file(dep_shp)
    
    # Convert shp to ras
    print '    Rasterising shapefile...'
    
    # Output BLR raster
    dep_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
               r'\GIS\Raster\dep_%s_30m.tif' % ser)

    cl.vec_to_ras(dep_shp, dep_tif, snap_tif, ser, -1, gdal.GDT_Int16)
    
    # Exceedance
    print '    Calculating exceedance...'
    
    # Read grids
    cl_grid, cl_ndv = cl.read_geotiff(cl_tif)
    dep_grid, dep_ndv = cl.read_geotiff(dep_tif)

    # Upcast to float32 for safe handling of negative values
    cl_grid = cl_grid.astype(np.float32)
    dep_grid = dep_grid.astype(np.float32)
   
    # Set ndv
    cl_grid[cl_grid==cl_ndv] = np.nan
    dep_grid[dep_grid==dep_ndv] = np.nan

    # Get total area of non-NaN from dep grid
    nor_area = np.count_nonzero(~np.isnan(dep_grid))*30.*30./1.E6

    # Apply scaling factor to CLs
    cl_grid = cl_grid*100.

    # Exceedance
    ex_grid = dep_grid - cl_grid
    del dep_grid, cl_grid  
    
    # Get total area exceeded
    ex_area = np.count_nonzero(ex_grid > 0)*30.*30./1.E6

    # Set <0 to 0
    ex_grid[ex_grid<0] = 0
    
    # Reset ndv
    ex_grid[np.isnan(ex_grid)] = -1

    # Downcast to int16 to save space
    ex_grid = ex_grid.round(0).astype(np.int16)
    
    # Append results
    data_dict['series'].append(ser)
    data_dict['nor_area_km2'].append(nor_area)
    data_dict['ex_area_km2'].append(ex_area)
    
    # Write output
    print '    Saving exceedance grid...'
    ex_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
              r'\GIS\Raster\exceed_%s_30m.tif' % ser)
    
    cl.write_geotiff(ex_grid, ex_tif, snap_tif, -1, gdal.GDT_Int16)
    
    print '    Done.'

# Build output df
ex_df = pd.DataFrame(data_dict)
ex_df['ex_pct'] = 100 * ex_df['ex_area_km2'] / ex_df['nor_area_km2']
ex_df.index = ex_df['series']
del ex_df['series']
ex_df = ex_df.round(0).astype(int)

# Save
out_csv = r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\vegetation\nor_prop_exceed.csv'
ex_df.to_csv(out_csv)

print 'Finished.'

Processing: Ndep78_82
    Building deposition shapefile...
    Rasterising shapefile...
    Calculating exceedance...

C:\Data\WinPython-64bit-2.7.10.3\python-2.7.10.amd64\lib\site-packages\ipykernel\__main__.py:78: RuntimeWarning: invalid value encountered in greater
C:\Data\WinPython-64bit-2.7.10.3\python-2.7.10.amd64\lib\site-packages\ipykernel\__main__.py:81: RuntimeWarning: invalid value encountered in less



    Saving exceedance grid...
    Done.
Processing: Ndep92_96
    Building deposition shapefile...
    Rasterising shapefile...
    Calculating exceedance...
    Saving exceedance grid...
    Done.
Processing: Ndep97_01
    Building deposition shapefile...
    Rasterising shapefile...
    Calculating exceedance...
    Saving exceedance grid...
    Done.
Processing: Ndep02_06
    Building deposition shapefile...
    Rasterising shapefile...
    Calculating exceedance...
    Saving exceedance grid...
    Done.
Processing: Ndep07_11
    Building deposition shapefile...
    Rasterising shapefile...
    Calculating exceedance...
    Saving exceedance grid...
    Done.
Processing: Ndep12_16
    Building deposition shapefile...
    Rasterising shapefile...
    Calculating exceedance...
    Saving exceedance grid...
    Done.
Finished.
Wall time: 15min 35s


### 1.6. Print national exceedance summary for vegetation

In [8]:
# Exceedance summary
ex_df

,ex_area_km2,nor_area_km2,ex_pct
series,,,
Ndep78_82,95122,320584,30
Ndep92_96,71470,320584,22
Ndep97_01,69485,320584,22
Ndep02_06,71065,320390,22
Ndep07_11,68475,320584,21
Ndep12_16,78717,320584,25


## 2. Water

The new methodology for water exceedances is developed [here](http://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_water.ipynb). The code below is copied from this notebook - see the original for further explanation.

### 2.1. Extract deposition data

**Note:** Remember to add the new `dep_series_id` to the query below.

In [9]:
# Get dep data
sql = ("SELECT a.blr, b.name AS series, c.name AS par, a.value AS dep "
       "FROM resa2.dep_blr_values a, "
       "resa2.dep_series_definitions b, "
       "resa2.air_parameter_definitions c "
       "WHERE a.parameter_id IN (1, 2, 4) "
       "AND a.dep_series_id IN (1, 2, 3, 4, 25, 26) "
       "AND a.parameter_id = c.parameter_id "
       "AND a.dep_series_id = b.dep_series_id")

dep_df = pd.read_sql(sql, engine)

# Tidy
dep_df['series'] = dep_df['series'].str[7:]
dep_df['par'] = dep_df['par'].str[:1]

# Group N(oks) and N(red)
dep_df = dep_df.groupby(['blr', 'series', 'par']).sum()

# Unstack
dep_df = dep_df.unstack('par')
dep_df.columns = dep_df.columns.get_level_values('par')

# Convert to meq/l
dep_df['N'] = dep_df['N'] / 14.
dep_df['S'] = dep_df['S']*2. / 32.06

# Unstack
dep_df = dep_df.unstack('series')

# Flatten col index
dep_df.columns = ['%s_%s' % (p, d) for p, d in zip(dep_df.columns.get_level_values('par'), 
                                                   dep_df.columns.get_level_values('series'))]

dep_df.head()

,N_1978-1982,N_1992-1996,N_1997-2001,N_2002-2006,N_2007-2011,N_2012-2016,S_1978-1982,S_1992-1996,S_1997-2001,S_2002-2006,S_2007-2011,S_2012-2016
blr,,,,,,,,,,,,
58006001,121.428571,126.512143,112.636429,NaN,97.391429,108.148571,99.812851,71.092327,54.825951,NaN,34.838428,26.530256
58006002,123.489286,89.075000,82.091429,85.187857,98.853571,100.497143,102.973175,47.747973,37.151591,32.061759,33.953837,24.578915
58006003,86.958571,83.657143,75.672857,82.670714,85.939286,87.740714,73.114161,46.873362,35.324392,30.809732,27.877105,21.721148
58006004,99.449286,93.594286,101.810714,93.010714,79.455714,84.100714,83.024953,49.577667,46.839676,34.863381,25.993762,20.893325
58006005,92.377143,93.052143,89.655000,74.112143,84.609286,91.922857,77.626326,52.674984,41.373674,27.650031,29.220836,22.189644


### 2.2. Get critical loads

In [10]:
# Get dep data
sql = ("SELECT blr, claoaavaroaa, eno3fl, clminn, clmaxnoaa, clmaxsoaa "
       "FROM resa2.cla")

cl_df = pd.read_sql(sql, engine)

# Set index
cl_df.index = cl_df['blr']
del cl_df['blr']

# Add CLSmin as 0
cl_df['clmins'] = 0

cl_df.head()

,claoaavaroaa,eno3fl,clminn,clmaxnoaa,clmaxsoaa,clmins
blr,,,,,,
60506013,46.689055,20.499,3.204124,59.153507,47.083568,0
60508003,47.240467,8.988,3.204124,70.544508,48.610725,0
60509002,48.396218,0.029,14.635616,100.955383,51.368535,0
60511002,11.156834,0.080,7.077605,27.656760,11.903564,0
60511006,34.764804,1.081,42.316831,121.992502,38.498971,0


### 2.3. Calculate exceedances

#### 2.3.1. Check data and join

In [11]:
# Check dfs are compatible
assert len(dep_df) == len(cl_df), 'Lengths of dataframes do not match.'
assert dep_df.index.is_unique, 'dep_df has duplicated BLR IDs.'
assert cl_df.index.is_unique, 'cl_df has duplicated BLR IDs.'

# Join
df = dep_df.join(cl_df, how='left')

# Fill NaN with 0
df.fillna(0, inplace=True)

df.head()

,N_1978-1982,N_1992-1996,N_1997-2001,N_2002-2006,N_2007-2011,N_2012-2016,S_1978-1982,S_1992-1996,S_1997-2001,S_2002-2006,S_2007-2011,S_2012-2016,claoaavaroaa,eno3fl,clminn,clmaxnoaa,clmaxsoaa,clmins
blr,,,,,,,,,,,,,,,,,,
58006001,121.428571,126.512143,112.636429,0.000000,97.391429,108.148571,99.812851,71.092327,54.825951,0.000000,34.838428,26.530256,28.533538,35.544,3.204124,40.514435,29.056767,0
58006002,123.489286,89.075000,82.091429,85.187857,98.853571,100.497143,102.973175,47.747973,37.151591,32.061759,33.953837,24.578915,32.029772,27.861,3.204124,43.473268,32.471152,0
58006003,86.958571,83.657143,75.672857,82.670714,85.939286,87.740714,73.114161,46.873362,35.324392,30.809732,27.877105,21.721148,20.888186,25.137,3.204124,29.564269,21.184960,0
58006004,99.449286,93.594286,101.810714,93.010714,79.455714,84.100714,83.024953,49.577667,46.839676,34.863381,25.993762,20.893325,18.914393,36.846,25.972633,49.911208,19.189395,0
58006005,92.377143,93.052143,89.655000,74.112143,84.609286,91.922857,77.626326,52.674984,41.373674,27.650031,29.220836,22.189644,20.463044,47.587,3.204124,30.057701,20.846989,0


#### 2.3.2. SSWC model

**Note:** Remember to update the `periods` list below.

In [12]:
# Define time periods
periods = ['1978-1982', '1992-1996', '1997-2001', '2002-2006', 
           '2007-2011', '2012-2016']

# Copy df
sswc_df = df.copy()

# Loop over periods
for per in periods:
    sswc_df['sswc_ex_%s' % per] = (sswc_df['S_%s' % per] + 
                                   sswc_df['eno3fl'] - 
                                   sswc_df['claoaavaroaa'])

# Get cols of interest
cols = ['sswc_ex_%s' % per for per in periods]
sswc_df = sswc_df[cols]

# Set values below 0 to 0
sswc_df[sswc_df<0] = 0

# Write to output
out_path = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\water\sswc_exceedance.csv')
sswc_df.to_csv(out_path)

sswc_df.head()

,sswc_ex_1978-1982,sswc_ex_1992-1996,sswc_ex_1997-2001,sswc_ex_2002-2006,sswc_ex_2007-2011,sswc_ex_2012-2016
blr,,,,,,
58006001,106.823313,78.102789,61.836413,7.010462,41.848890,33.540718
58006002,98.804403,43.579201,32.982819,27.892987,29.785065,20.410143
58006003,77.362975,51.122176,39.573206,35.058546,32.125919,25.969962
58006004,100.956560,67.509274,64.771282,52.794988,43.925369,38.824932
58006005,104.750281,79.798940,68.497630,54.773987,56.344792,49.313600


#### 2.3.3. FAB model

**Note:** Remember to update the `periods` list below.

In [13]:
# Define time periods
periods = ['1978-1982', '1992-1996', '1997-2001', '2002-2006', 
           '2007-2011', '2012-2016']

# Plot folder
png_fold = r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\water\CLF_PNGs'

# Copy df
fab_df = df.copy()

# Loop over periods
for per in periods:
    print 'Processing %s...' % per
    
    # Containers for data
    fab_ex_n = []
    fab_ex_s = []
    fab_ex_reg = []
    
    # Loop over rows
    for idx, row in df.iterrows():
        # Calc exceedance
        ex_n, ex_s, reg_id = cl.exceed_ns_icpm(row['clminn'], row['clmaxnoaa'],
                                               row['clmins'], row['clmaxsoaa'],
                                               row['N_%s' % per], row['S_%s' % per])
        
        # Add to result
        fab_ex_n.append(ex_n)
        fab_ex_s.append(ex_s)
        fab_ex_reg.append(reg_id)
        
        # Plot
#        out_png = os.path.join(png_fold, 'blr_%s_%s.png' % (idx, per))
#        cl.plot_critical_loads_func(row['clminn'], row['clmaxnoaa'],
#                                    row['clmins'], row['clmaxsoaa'],
#                                    [row['N_%s' % per],], [row['S_%s' % per]],
#                                    title='%s (%s)' % (idx, per), save_png=True, 
#                                    png_path=out_png)
#        plt.close()
        
    # Add to df
    fab_df['fab_ex_n_%s' % per] = fab_ex_n
    fab_df['fab_ex_s_%s' % per] = fab_ex_s
    fab_df['fab_ex_reg_%s' % per] = fab_ex_reg
    
    # Calc total ex = ex_n + ex_s
    fab_df['fab_ex_%s' % per] = fab_df['fab_ex_n_%s' % per] + fab_df['fab_ex_s_%s' % per]

# Get cols of interest
cols = [i for i in fab_df.columns if i[:3]=='fab']
fab_df = fab_df[cols]

# Set values below 0 to 0
fab_df[fab_df<0] = 0

# Write to output
out_path = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\water\fab_exceedance.csv')
fab_df.to_csv(out_path)

print 'Finished.'

fab_df.head()

Processing 1978-1982...
Processing 1992-1996...
Processing 1997-2001...
Processing 2002-2006...
Processing 2007-2011...
Processing 2012-2016...
Finished.


,fab_ex_n_1978-1982,fab_ex_s_1978-1982,fab_ex_reg_1978-1982,fab_ex_1978-1982,fab_ex_n_1992-1996,fab_ex_s_1992-1996,fab_ex_reg_1992-1996,fab_ex_1992-1996,fab_ex_n_1997-2001,fab_ex_s_1997-2001,...,fab_ex_reg_2002-2006,fab_ex_2002-2006,fab_ex_n_2007-2011,fab_ex_s_2007-2011,fab_ex_reg_2007-2011,fab_ex_2007-2011,fab_ex_n_2012-2016,fab_ex_s_2012-2016,fab_ex_reg_2012-2016,fab_ex_2012-2016
blr,,,,,,,,,,,,,,,,,,,,,
58006001,80.914136,99.812851,2,180.726987,85.997708,71.092327,2,157.090034,72.121993,54.825951,...,0,0.000000,56.876993,34.838428,2,91.715421,67.634136,26.530256,2,94.164392
58006002,81.844111,101.499086,3,183.343197,45.601732,47.747973,2,93.349704,38.618160,37.151591,...,2,73.776348,55.380303,33.953837,2,89.334140,57.023874,24.578915,2,81.602789
58006003,58.224030,72.447331,3,130.671361,54.092874,46.873362,2,100.966237,46.108588,35.324392,...,2,83.916177,56.375017,27.877105,2,84.252122,58.176446,21.721148,2,79.897593
58006004,59.897117,74.721047,3,134.618163,43.683077,49.577667,2,93.260744,51.899506,46.839676,...,2,77.962887,29.544506,25.993762,2,55.538268,34.189506,20.893325,2,55.082831
58006005,62.319442,77.626326,2,139.945768,62.994442,52.674984,2,115.669427,59.597299,41.373674,...,2,71.704474,54.551585,29.220836,2,83.772421,61.865157,22.189644,2,84.054801


#### 2.3.4. Area exceeded

In [14]:
# Read BLR grid
in_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
          r'\Shapefiles\blrgrid_uten_grums.shp')
blr_df = gpd.read_file(in_shp)

# Convert to 'normal' df
del blr_df['geometry']
blr_df = pd.DataFrame(blr_df)

# Get total area of Norway by summing cells
nor_area = blr_df['area_m2'].sum()/1E6

blr_df.head()

,BLR,area_m2
0,58005004,5.410322e+07
1,58006001,1.939285e+08
2,58006002,2.044114e+08
3,58006003,2.043394e+08
4,58006004,2.042695e+08


##### 2.3.4.1. SSWC

In [15]:
# Container for output
data_dict = {'period':[],
             'area_exceeded_km2':[],
             'area_total_km2':[]}

# Caclulate area exceeded
for col in sswc_df.columns:
    per = col[-9:]
    
    # Get BLRs exceeded
    ex_blr = sswc_df[sswc_df[col]>0].index
    
    # Get area exceeded
    area_ex = blr_df['area_m2'][blr_df['BLR'].isin(ex_blr)].sum()/1E6
    
    # Add to output
    data_dict['period'].append(per)
    data_dict['area_exceeded_km2'].append(area_ex)
    data_dict['area_total_km2'].append(nor_area)
    
# Build df
sswc_areas = pd.DataFrame(data_dict)
sswc_areas.index = sswc_areas['period']
del sswc_areas['period']

# Pct exceed
sswc_areas['pct_exceed'] = 100*sswc_areas['area_exceeded_km2']/nor_area

# Tidy
sswc_areas = sswc_areas.round(0).astype(int)

# Write output
out_csv = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\water'
           r'\sswc_area_exceeded.csv')
sswc_areas.to_csv(out_csv)

sswc_areas

,area_exceeded_km2,area_total_km2,pct_exceed
period,,,
1978-1982,100964,322183,31
1992-1996,55695,322183,17
1997-2001,42949,322183,13
2002-2006,35351,322183,11
2007-2011,26773,322183,8
2012-2016,22128,322183,7


##### 2.3.4.2. FAB

**Note:** Remember to update the `periods` list below.

In [16]:
# Get cols for overall exceedance
periods = ['1978-1982', '1992-1996', '1997-2001', '2002-2006', 
           '2007-2011', '2012-2016']
cols = ['fab_ex_%s' % i for i in periods]
fab = fab_df[cols]

# Container for output
data_dict = {'period':[],
             'area_exceeded_km2':[],
             'area_total_km2':[]}

# Caclulate area exceeded
for col in fab.columns:
    per = col[-9:]
    
    # Get BLRs exceeded
    ex_blr = fab[fab[col]>0].index
    
    # Get area exceeded
    area_ex = blr_df['area_m2'][blr_df['BLR'].isin(ex_blr)].sum()/1E6
    
    # Add to output
    data_dict['period'].append(per)
    data_dict['area_exceeded_km2'].append(area_ex)
    data_dict['area_total_km2'].append(nor_area)
    
# Build df
fab_areas = pd.DataFrame(data_dict)
fab_areas.index = fab_areas['period']
del fab_areas['period']

# Pct exceed
fab_areas['pct_exceed'] = 100*fab_areas['area_exceeded_km2']/nor_area

# Tidy
fab_areas = fab_areas.round(0).astype(int)

# Write output
out_csv = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\water'
           r'\fab_area_exceeded.csv')
fab_areas.to_csv(out_csv)

fab_areas

,area_exceeded_km2,area_total_km2,pct_exceed
period,,,
1978-1982,135904,322183,42
1992-1996,94119,322183,29
1997-2001,79389,322183,25
2002-2006,75100,322183,23
2007-2011,65009,322183,20
2012-2016,64759,322183,20


## 3. Soil

The soil calculations use the "old" method - see e-mail from Kari received 01/11/2017 at 13.47. The equation is simply

$$E_{soil} = S_{dep} - CL_{soil}$$

### 3.1. Get deposition data from database

**Note:** Remember to add the new `dep_series_id` to the query below, as well as a new name in `df.columns`.

In [17]:
# Get all dep values for non-marine S
# in mg-S/m2/year
sql = ("SELECT blr, dep_series_id as series, value as dep "
       "FROM resa2.dep_blr_values "
       "WHERE parameter_id = 4 "
       "AND dep_series_id IN (1, 2, 3, 4, 25, 26)")

dep_df = pd.read_sql(sql, engine)

# Reshape and tidy
dep_df = dep_df.pivot(index='blr', columns='series', values='dep')
dep_df.columns.name = ''
dep_df.columns = ['Sdep78_82', 'Sdep92_96', 'Sdep97_01', 'Sdep02_06', 
                  'Sdep07_11', 'Sdep12_16']

dep_df.head()

,Sdep78_82,Sdep92_96,Sdep97_01,Sdep02_06,Sdep07_11,Sdep12_16
blr,,,,,,
58006001,1600.00,1139.61,878.86,NaN,558.46,425.28
58006002,1650.66,765.40,595.54,513.95,544.28,394.00
58006003,1172.02,751.38,566.25,493.88,446.87,348.19
58006004,1330.89,794.73,750.84,558.86,416.68,334.92
58006005,1244.35,844.38,663.22,443.23,468.41,355.70


### 3.2. Get critical loads

**Note:** Parameter 86 (`CL-Soil-N`) is strangely named, but the description implies it is the critical load for S.

**Note 2:** There is one cell where the critical loads is negative. This is filtered out from the calculations - see e-mail from Espen received 16/11/2017 at 13.04.

In [18]:
# Get all CLs for S
# in meq/m2/year
sql = ("SELECT blr, xvalue as crit_ld "
       "FROM resa2.talegren_values "
       "WHERE talegren_paramid = 86")

cl_df = pd.read_sql(sql, engine)
cl_df.index = cl_df['blr']
del cl_df['blr']

# Convert to mg-S/m2/yr
cl_df['crit_ld'] = cl_df['crit_ld']*32.06 / 2.

# Remove negative CL
cl_df = cl_df.query('crit_ld >= 0')

cl_df.head()

,crit_ld
blr,
58006004,1818.923277
58006007,3237.448624
58006008,3095.109867
58006012,3443.772948
58006015,3062.583133


### 3.3. Calculate exceedances

In [19]:
# Join
df = dep_df.join(cl_df)

# Exceedance
for col in dep_df.columns:
    per = col[-5:]
    df['Ex%s' % per] = df[col] - df['crit_ld']
    
# Get cols of interest
ex_df = df[[i for i in df.columns if i[:2]=='Ex']].copy()
ex_df.dropna(how='any', inplace=True)

ex_df.head()

,Ex78_82,Ex92_96,Ex97_01,Ex02_06,Ex07_11,Ex12_16
blr,,,,,,
58006004,-488.033277,-1024.193277,-1068.083277,-1260.063277,-1402.243277,-1484.003277
58006007,-1920.338624,-2203.388624,-2392.638624,-2585.288624,-2716.118624,-2833.648624
58006008,-1790.899867,-2316.929867,-2217.989867,-2389.629867,-2633.669867,-2735.589867
58006012,-1590.972948,-2262.992948,-2545.382948,-2790.722948,-2940.882948,-3064.252948
58006015,-1667.103133,-1763.793133,-2006.503133,-2318.493133,-2481.303133,-2620.853133


### 3.4. Area exceeded

In [20]:
# Read BLR grid
in_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
          r'\Shapefiles\blrgrid_uten_grums.shp')
blr_df = gpd.read_file(in_shp)

# Convert to 'normal' df
del blr_df['geometry']
blr_df = pd.DataFrame(blr_df)

# Get total area of Norway by summing cells
nor_area = blr_df['area_m2'].sum()/1E6

blr_df.head()

,BLR,area_m2
0,58005004,5.410322e+07
1,58006001,1.939285e+08
2,58006002,2.044114e+08
3,58006003,2.043394e+08
4,58006004,2.042695e+08


In [21]:
# Container for output
data_dict = {'period':[],
             'area_exceeded_km2':[],
             'area_total_km2':[]}

# Caclulate area exceeded
for col in ex_df.columns:
    per = col[-5:]
    
    # Get BLRs exceeded
    ex_blr = ex_df[ex_df[col]>0].index
    
    # Get area exceeded
    area_ex = blr_df['area_m2'][blr_df['BLR'].isin(ex_blr)].sum()/1E6
    
    # Add to output
    data_dict['period'].append(per)
    data_dict['area_exceeded_km2'].append(area_ex)
    data_dict['area_total_km2'].append(nor_area)
    
# Build df
areas = pd.DataFrame(data_dict)
areas.index = areas['period']
del areas['period']

# Pct exceed
areas['pct_exceed'] = 100*areas['area_exceeded_km2']/nor_area

# Tidy
areas = areas.round(2)

# Write output
out_csv = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\soils'
           r'\soils_area_exceeded.csv')
areas.to_csv(out_csv)

areas

,area_exceeded_km2,area_total_km2,pct_exceed
period,,,
78_82,1379.34,322183.39,0.43
92_96,199.02,322183.39,0.06
97_01,0.00,322183.39,0.00
02_06,0.00,322183.39,0.00
07_11,0.00,322183.39,0.00
12_16,0.00,322183.39,0.00
